In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


dataset = load_dataset("arbml/Ashaar_dataset") 



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/212499 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'poem title': 'أصبح الملك للذي فطر الخلق',
 'poem meter': 'الخفيف',
 'poem verses': ['أَصبَحَ المُلك لِلَّذي فَطر الخَل',
  'قَ بِتَقديرٍ للعَزيز العَليمِ',
  'غافر الذَنب للمسيءِ بِعَفوٍ',
  'قابل التَوب ذي العَطاء العَميمِ',
  'مُرسل المُصطَفى البَشير ِلَينا',
  'رَحمة مِنهُ بِالكَلام القَديمِ',
  'رَبَنا رَبّنا ِلَيكَ أَنينا',
  'فَأَجرنا مِن حَر نار الجَحيمِ',
  'وَاكفِنا شَرّ ما نَخاف بِلُطفٍ',
  'يا عَظيماً يَرجى لِكُل عَظيمِ',
  'وَتَقبل أَعمالَنا وَاعفُ عَنا',
  'وَأَنلنا دُخول دار النَعيمِ',
  'بِنَبي بَعثَتهُ فَهَدانا',
  'لِصِراط مِن الهُدى مُستَقيمِ',
  'وَبِمَن نَحنُ في حِماهُ مَدى الدَهر',
  'أَخيهِ يَحيى الحصور الكَريمِ',
  'أَدرك أَدرك قَوماً أَتوا بافتقار',
  'وَاِنكِسار وَمَدمَع مَسجومِ',
  'شَهدت أَرواحَهُم أَنكَ اللَهُ',
  'وَجاءوا بِكُل قَلبٍ سَليم'],
 'poem theme': 'قصيدة دينية',
 'poem url': 'https://www.aldiwan.net/poem16182.html',
 'poet name': 'الامير منجك باشا',
 'poet description': 'منجك بن محمد بن منجك بن ابي بكر بن عبد القادر بن ابراهيم بن منجك اليوسفي ال

In [24]:
import pyarabic.araby as araby

DEFAULT_ARABIC_SYSTEM_PROMPT = """
Generate a poem based on the following title, and the given era:
""".strip()
def generate_arabic_training_prompt(
    example: dict,
    field: str = 'poem',
    train = True,
) -> str:
    

    verses = example['poem verses']
    era = example['poet era']
    title = example['poem title']
    
    if not era:
        example['poet era'] = "العصر الجاهلي"
        era = example['poet era']
    if not title:
        example['poem title'] = "العصر الجاهلي"
        title = example['poem title']
    
    

    poem = ''
    for verse in verses:

        verse = verse.strip()
        verse = araby.strip_diacritics(verse)
        poem += verse
        poem += '\n'


    example['poem'] = poem

    if train:
        example[field] = f"""### Instruction: {DEFAULT_ARABIC_SYSTEM_PROMPT}

    ### Input:
    {title.strip()} , {era.strip()}

    ### Response:
    {poem}
    """.strip()


    else:

        example[field] =  f"""### Instruction: {DEFAULT_ARABIC_SYSTEM_PROMPT}

    ### Input:
    {title.strip()} , {era.strip()}
    """.strip()  

    return example

In [30]:
def get_arabic_datasets(dataset_name = "arbml/Ashaar_dataset",field = 'prompt'):

    train_dataset = load_dataset(dataset_name,split='train[:80%]')
    val_dataset = load_dataset(dataset_name,split='train[80%:90%]')
    test_dataset = load_dataset(dataset_name,split='train[90%:]')




    train_dataset = train_dataset.map(generate_arabic_training_prompt, fn_kwargs={"field": field, "train" : True})
    val_dataset = val_dataset.map(generate_arabic_training_prompt, fn_kwargs={"field": field , "train" : False})
    test_dataset = test_dataset.map(generate_arabic_training_prompt, fn_kwargs={"field": field , "train" : False})


    train_dataset = train_dataset.select_columns([field, 'poem'])
    val_dataset = val_dataset.select_columns([field, 'poem'])
    test_dataset = test_dataset.select_columns([field, 'poem'])
    

    return train_dataset, val_dataset, test_dataset

In [31]:
train_dataset, val_dataset, test_dataset = get_arabic_datasets(field = 'prompt')

In [32]:
train_dataset[0]

{'prompt': '### Instruction: Generate a poem based on the following title, and the given era:\n\n    ### Input:\n    أصبح الملك للذي فطر الخلق , العصر العثماني\n\n    ### Response:\n    أصبح الملك للذي فطر الخل\nق بتقدير للعزيز العليم\nغافر الذنب للمسيء بعفو\nقابل التوب ذي العطاء العميم\nمرسل المصطفى البشير لينا\nرحمة منه بالكلام القديم\nربنا ربنا ليك أنينا\nفأجرنا من حر نار الجحيم\nواكفنا شر ما نخاف بلطف\nيا عظيما يرجى لكل عظيم\nوتقبل أعمالنا واعف عنا\nوأنلنا دخول دار النعيم\nبنبي بعثته فهدانا\nلصراط من الهدى مستقيم\nوبمن نحن في حماه مدى الدهر\nأخيه يحيى الحصور الكريم\nأدرك أدرك قوما أتوا بافتقار\nوانكسار ومدمع مسجوم\nشهدت أرواحهم أنك الله\nوجاءوا بكل قلب سليم',
 'poem': 'أصبح الملك للذي فطر الخل\nق بتقدير للعزيز العليم\nغافر الذنب للمسيء بعفو\nقابل التوب ذي العطاء العميم\nمرسل المصطفى البشير لينا\nرحمة منه بالكلام القديم\nربنا ربنا ليك أنينا\nفأجرنا من حر نار الجحيم\nواكفنا شر ما نخاف بلطف\nيا عظيما يرجى لكل عظيم\nوتقبل أعمالنا واعف عنا\nوأنلنا دخول دار النعيم\nبنبي بعثته فهدانا\nلصر